In [1]:
import os

In [2]:
%pwd

'f:\\Files\\DS&ML\\FraudGuard\\notebooks'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Files\\DS&ML\\FraudGuard'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    bucket: str
    region: str
    data_path: str
    download_data: Path

In [7]:
from FraudGuard.utils.helpers import *
from FraudGuard.constants import *

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH,
        schema_filepath = SCHEMA_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            bucket=config.bucket,
            region=config.region,
            data_path=config.data_path,
            download_data=Path(config.download_data)
        )

        return data_ingestion_config

In [9]:
import urllib.request as request
import zipfile
import gdown
from FraudGuard import logger

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        # Use download_data as the local path for the CSV file
        local_csv_path = self.config.download_data
        if not os.path.exists(local_csv_path):
            success = download_from_s3(
                bucket=self.config.bucket,
                s3_path=self.config.data_path,
                local_path=local_csv_path,
                aws_region=self.config.region
            )
            if success:
                logger.info(f"Downloaded file: {local_csv_path}")
            else:
                raise Exception(f"Failed to download {self.config.data_path} from S3")
        else:
            logger.info(f"File already exists: {local_csv_path} ({get_size(local_csv_path)})")

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    
except Exception as e:
    logger.error(f"Error: {e}")

[2025-06-17 12:34:28,715: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-06-17 12:34:28,727: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-06-17 12:34:28,734: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-06-17 12:34:28,734: INFO: helpers: created directory at: artifacts]
[2025-06-17 12:34:28,734: INFO: helpers: created directory at: artifacts/data_ingestion]
[2025-06-17 12:34:28,762: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]
[2025-06-17 12:34:33,128: INFO: helpers: Downloaded s3://fraudguard/Fraud-data.csv to artifacts\data_ingestion\Fraud-data.csv]
[2025-06-17 12:34:33,134: INFO: 3942098000: Downloaded file: artifacts\data_ingestion\Fraud-data.csv]
